<a href="https://colab.research.google.com/github/Afroza2/KaggleProjects/blob/WiDS_2022/LSTM_Third_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import numpy as np
from scipy.stats import randint
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

## Deep Learning
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.optimizers import gradient_descent_v2 ## Kaggle notebook mentions it as (SGD)
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

## Importing the Data and Processing

In [33]:
from google.colab import files
files.upload()



Saving Third_predict.csv to Third_predict (1).csv


In [34]:
df = pd.read_csv('Third_predict (1).csv','\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

In [ ]:
df.info()

In [37]:
df=df.drop('Unnamed: 0',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [38]:
df.shape

(85462, 64)

In [ ]:
df.describe()

In [ ]:
df.columns

## LSTM Begins

In [40]:
def time_series_to_supervised(data, n_lag=1, n_fut=1, selLag=None, selFut=None, dropnan=True):
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df1 = pd.DataFrame(data)
    origNames = df.columns
    cols, names = list(), list()
    # include all current period attributes
    cols.append(df.shift(0))
    names += [('%s' % origNames[j]) for j in range(n_vars)]
 
	# lag any past period attributes (t-n_lag,...,t-1)
    n_lag = max(0, n_lag)  # force valid number of lag periods
    for i in range(n_lag, 0, -1):
        suffix= '(t-%d)' % i
        if (None == selLag):   # copy all attributes from PRIOR periods?
            cols.append(df.shift(i))
            names += [('%s%s' % (origNames[j], suffix)) for j in range(n_vars)]
        else:
            for var in (selLag):
                cols.append(df[var].shift(i))
                names+= [('%s%s' % (var, suffix))]
 
    # include future period attributes (t+1,...,t+n_fut)
    n_fut = max(n_fut, 0)  # force valid number of future periods to shift back
    for i in range(1, n_fut + 1):
        suffix= '(t+%d)' % i
        if (None == selFut):  # copy all attributes from future periods?
            cols.append(df.shift(-i))
            names += [('%s%s' % (origNames[j], suffix)) for j in range(n_vars)]
        else:  # copy only selected future attributes
            for var in (selFut):
                cols.append(df[var].shift(-i))
                names += [('%s%s' % (var, suffix))]
    # combine everything
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values introduced by lagging
    if dropnan:
        agg.dropna(inplace=True)
    return agg


  


In [41]:

##values=df.values
#normalizing features
#scaler=MinMaxScaler(feature_range=(0,1)) #scaling all the features in range of [0,1]
#scaled=scaler.fit_transform(values)
#supervised learning
reframed=time_series_to_supervised(df,10,1, selFut=['site_eui'])

#drop columns we don't want want to predict
#reframed.drop(reframed.columns[65:],axis=1,inplace=True)
print(reframed.head())


    Year_Factor  State_Factor  ...  site_eui(t-1)  site_eui(t+1)
10          1.0           0.0  ...           29.0           54.0
11          1.0           0.0  ...           30.0           78.0
12          1.0           0.0  ...           54.0          537.0
13          1.0           0.0  ...           78.0            2.0
14          1.0           0.0  ...          537.0           24.0

[5 rows x 705 columns]


In [42]:
reframed.shape

(85451, 705)

In [ ]:
for i in reframed.columns:
    print (i)

In [43]:
n_features_in = 63
n_features_out = 1
n_lags_in = 10
n_lags_out = 1

n_obs_in = (n_features_in * n_lags_in)
n_obs_out = (n_features_out * n_lags_out)
print ('Number of observations in:',n_obs_in)
print ('Number of predictions out:', n_obs_out)

# Creating X, y from data_test
X = reframed.values[:, :n_obs_in]
y = reframed.values[:, -n_obs_out:]
print ('\nShape of X:',X.shape)
print ('Shape of y:', y.shape)

Number of observations in: 630
Number of predictions out: 1

Shape of X: (85451, 630)
Shape of y: (85451, 1)


In [ ]:
if True:
    for i in reframed.columns[:n_obs_in]:
        print (i)

In [ ]:
if False:
    for i in data_test.columns[-n_obs_out:]:
        print (i)

In [44]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X)
y_scaler = MinMaxScaler(feature_range=(0,1)).fit(y)

In [45]:
n_train = 75000
n_features = 63
n_lag = 10
n_in, n_out = 1, 1

# split X, y into train/test
X_train, y_train = X[:n_train, :], y[:n_train, :]
X_test, y_test = X[n_train:, :], y[n_train:, :]
print('Shape of splits:')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Shape of splits:
(75000, 630) (75000, 1) (10451, 630) (10451, 1)


In [46]:
X_train, y_train = X_scaler.transform(X_train), y_scaler.transform(y_train)
X_test, y_test = X_scaler.transform(X_test), y_scaler.transform(y_test)

In [47]:
X_train = X_train.reshape((X_train.shape[0], n_lag, n_features))
X_test = X_test.reshape((X_test.shape[0], n_lag, n_features))
print('Re-shaped for LSTM:')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Re-shaped for LSTM:
(75000, 10, 63) (75000, 1) (10451, 10, 63) (10451, 1)


In [48]:
senjuti_lite=Sequential()
senjuti_lite.add(LSTM(512, activation='relu', return_sequences=True, input_shape=((X_train.shape[1]), X_train.shape[2])))
senjuti_lite.add(LSTM(512,activation='relu'))
senjuti_lite.add(Dense(n_out))
senjuti_lite.add(Dropout(0.2))
sgd=tf.keras.optimizers.SGD(learning_rate=0.001)
senjuti_lite.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
history_test = senjuti_lite.fit(X_train, y_train, 
                    epochs=12, batch_size=32, 
                    validation_data=(X_test, y_test), 
                    verbose=1, shuffle=False)

In [47]:
senjuti_lite.save('halkasenjutilite.h5')

In [49]:
#make the prediction
yhat_entire = senjuti_lite.predict(X_test)

#print the shape
print (yhat_entire.shape)

(10451, 1)


In [50]:
inv_yhat_entire = y_scaler.inverse_transform(yhat_entire)

print ('inv_yhat_entire shape:',inv_yhat_entire.shape)

inv_yhat_entire shape: (10451, 1)


In [51]:
df55=pd.DataFrame(inv_yhat_entire)

In [52]:
df55

,0
0,33.820297
1,29.982834
2,30.244356
3,30.493784
4,30.493975
...,...
10446,27.143103
10447,30.384779
10448,29.794027
10449,26.648569


In [53]:
df55.to_csv('Fourth_Predict_site eui.csv','\t')